# Premade Models

Although it is possible to define your own custom model (see [custom models](custom_models.ipynb)), most use cases will want a common set of emisisons with a common set of properties. To avoid unnecessarily redefining common models every time Synthesizer is used we provide some premade models which can be used "out of the box" or as the foundation for constructing more complex models.

These premade models can be imported directly from the `emission_models` submodule which also defines several lists detailing all available models.

In [ ]:
from synthesizer.emission_models import (
    AGN_MODELS,
    COMMON_MODELS,
    PREMADE_MODELS,
    STELLAR_MODELS,
)

print(STELLAR_MODELS)
print(AGN_MODELS)
print(COMMON_MODELS)
print(PREMADE_MODELS)

As you can see in the code above, there are ``STELLAR_MODELS``, ``AGN_MODELS``, and ``COMMON_MODELS``. In the following sections we'll detail each of these.

## Stellar Emission Models

Before we detail each stellar model we'll first need a grid to pass to each model.

In [ ]:
import numpy as np
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Blackbody
from synthesizer.grid import Grid
from unyt import Hz, Myr, dimensionless, erg, kelvin, s

# Get the grid which we'll need for extraction
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

### IncidentEmission

Incident emission is the pure stellar emission which is fed into photoionisation modelling. An incident emission model defines the extraction of the ``"incident"`` SPS spectra.

In [ ]:
from synthesizer.emission_models import IncidentEmission

incident = IncidentEmission(grid=grid)
print(incident)

### LineContinuumEmission

Line continuum emission is, unsurpirsingly, the line continuum emitted from the stellar birth cloud of young stars. A line continuum model defines the extraction of the ``"linecont"`` reprocessed SPS spectra.

Here we will also use a mask because line continuum should be isolated to young stars still surrounded by their birth cloud.

In [ ]:
from synthesizer.emission_models import LineContinuumEmission

line_cont = LineContinuumEmission(
    grid, fesc=0.8, mask_attr="ages", mask_op="<", mask_thresh=10 * Myr
)
print(line_cont)

### TransmittedEmission 

Transmitted emission is the incident emission that is transmitted through the gas in photoionisation modelling. Unlike incident emission, transmitted emission has little flux below the Lyman-limit, since it has been absorbed by the gas (depending on ``fesc``). A transmitted model defines the extraction of the ``"transmitted"`` reprocessed SPS spectra with some escape fraction.

In [ ]:
from synthesizer.emission_models import TransmittedEmission

transmitted = TransmittedEmission(grid=grid, fesc=0.1)
print(transmitted)

### EscapedEmission

Escaped emission is the portion of the incident emission which fully escapes without being effected by the gas. An escaped model defines the extraction of the ``"transmitted"`` reprocessed SPS spectra with some escape fraction (i.e. ``escaped = fesc * incident``).

In [ ]:
from synthesizer.emission_models import EscapedEmission

escaped = EscapedEmission(grid, fesc=0.1)
print(escaped)

Notice here that the "escape fraction" for the escaped model is ``1 - fesc`` because escaped is the inverse of transmitted.

### NebularContinuumEmission

Nebular continuum emission is the continuum emission coming directly from young stars birth cloud. A nebular continuum model defines the extraction of the ``"nebular_continuum"`` reprocessed SPS spectra.

Like ``LineContinuum`` we will also use a mask here because nebular continuum should be isolated to young stars still surrounded by their birth cloud.


In [ ]:
from synthesizer.emission_models import NebularContinuumEmission

nebular_cont = NebularContinuumEmission(
    grid, fesc=0.2, mask_attr="ages", mask_op="<", mask_thresh=10 * Myr
)
print(nebular_cont)

### NebularEmission

Nebular emission is the emission from young stars birth cloud. Unlike other models we've shown so far ``NebularEmission`` comes in two forms depending on whether a lyman alpha escape fraction less than 1.0 is passed. If a lyman alpha escape fraction is passed then ``NebularEmission`` is the combination of ``LineContinuumEmission`` and ``NebularContinuum``, if not it is an extraction of the ``"nebular"`` reprocessed SPS spectra.

Again, like ``LineContinuumEmission`` and ``NebularContinuum`` we'll include a mask for young stars.

In [ ]:
from synthesizer.emission_models import NebularEmission

nebular_extract = NebularEmission(grid)
print(nebular_extract)
print()

nebular_combined = NebularEmission(
    grid,
    fesc=0.1,
    fesc_ly_alpha=0.8,
    mask_attr="ages",
    mask_op="<",
    mask_thresh=10 * Myr,
)
print(nebular_combined)
nebular_combined.plot_emission_tree()

In the above we had our first model with children. For more complex models like this we can plot the "emission tree" to visualise the model. When the model is eventually used to generate emission, the model will be traversed bottom to top (breadth first) generating each spectra.

### ReprocessedEmission 

Reprocessed emission is the stellar emission which has been reprocessed by gas (i.e. the combination of the reprocessed spectra we saw above). A reprocessed model defines the combination of ``TransmittedEmission`` and ``NebularEmission``.

In [ ]:
from synthesizer.emission_models import ReprocessedEmission

reprocessed = ReprocessedEmission(grid, fesc=0.1, fesc_ly_alpha=0.8)
print(reprocessed)
reprocessed.plot_emission_tree()

### IntrinsicEmission

Intrinsic emission is the total stellar emission including all reprocessing prior to dust attenuation. Effectively intrinsic emisison is the emission that is "incident" on the dust distribution. An intrinsic model defines the combination of ``EscapedEmission`` and ``ReprocessedEmission``.

In [ ]:
from synthesizer.emission_models import IntrinsicEmission

intrinsic = IntrinsicEmission(grid, fesc=0.1, fesc_ly_alpha=1.0)
print(intrinsic)
intrinsic.plot_emission_tree()

### EmergentEmission 

Emergent emission is the stellar emission reprocessed by gas attenuated by dust. We haven't seen the Attenuation model yet since this is a "common" model and will be covered below. An emergent model is a combination of ``EscapedEmission`` and ``AttenuatedEmission`` (see below).

In [ ]:
from synthesizer.emission_models import EmergentEmission

emergent = EmergentEmission(
    grid=grid,
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=reprocessed,
    tau_v=0.67,
    fesc=0.2,
)
print(emergent)
emergent.plot_emission_tree()

### TotalEmission

Total emission is the combined attenuated emission from the stellar population along with the thermal emission from dust. Once again, dust emission is a common model and will be covered below. A total model is a combination of ``DustEmission`` and ``EmergentEmission``.


In [ ]:
from synthesizer.emission_models import TotalEmission

total = TotalEmission(
    grid=grid,
    dust_curve=PowerLaw(slope=-1),
    tau_v=0.67,
    fesc=0.2,
    fesc_ly_alpha=0.7,
    dust_emission_model=Blackbody(temperature=100 * kelvin),
)
print(total)
total.plot_emission_tree()

Note that omitting `dust_emission_model` when initialising `TotalEmission` will result in a simpler model where `total == emergent`.

### CharlotFall2000

In addition to the simpler models shown above we also provide 3 more complex models which produce more specialised emissions.

The first of these follows Charlot&Fall+2000 including emission split into young and old populations (with the threshold defined by ``age_pivot``) with dust attenuation specific to each population.

In [ ]:
from synthesizer.emission_models import CharlotFall2000

cf_model = CharlotFall2000(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-0.7),
    dust_curve_nebular=PowerLaw(slope=-1.3),
    age_pivot=7 * dimensionless,
    dust_emission_ism=Blackbody(temperature=50 * kelvin),
    dust_emission_nebular=Blackbody(temperature=100 * kelvin),
)
print(cf_model)
cf_model.plot_emission_tree(fontsize=6)

If we omit the dust emission model then we get a simpler model with a root at ``"emergent"``.

In [ ]:
cf_model = CharlotFall2000(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-0.7),
    dust_curve_nebular=PowerLaw(slope=-1.3),
    age_pivot=7 * dimensionless,
)
cf_model.plot_emission_tree()

It's also possible to plot a subtree within a model by passing the root of the subtree. This is particularly helpful for models like `CharlotFall2000` which define a lot of extra spectra that don't necessarily appear in the main tree depending on the exact inputs. You can see this in the plots below which show the trees defining the "extra" spectra available when using ``CharlotFall2000`` (and indeed ``PacmanEmission`` and ``BimodelPacmanEmission`` which we'll cover shortly).

In [ ]:
cf_model.plot_emission_tree(root="incident")

In [ ]:
cf_model.plot_emission_tree(root="transmitted")

In [ ]:
cf_model.plot_emission_tree(root="intrinsic")

In [ ]:
cf_model.plot_emission_tree(root="attenuated")

### PacmanEmission

We also implement a generalised form of Charlot&Fall+2000 which, unlike ``CharlotFall2000``, also accounts for escape fractions. We call this model "Pacman". Pacman calculates dust attenuated spectra including an escape fraction and variable Lyman-alpha transmission. In this model some fraction (fesc) of the pure stellar emission is able to completely escaped the galaxy without reprocessing by gas or dust. The rest is assumed to be reprocessed by both gas and a screen of dust. 

The Pacman model has two forms, the first is the ``PacmanEmission`` model which does not differentiate between young and old populations.

In [ ]:
from synthesizer.emission_models import PacmanEmission

# Simple Pacman with dust emission
simple_pc_model = PacmanEmission(
    grid=grid,
    tau_v=0.7,
    dust_curve=PowerLaw(slope=-1.3),
    dust_emission=Blackbody(temperature=100 * kelvin),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)
simple_pc_model.plot_emission_tree()

Again, omitting the ``dust_emission_model`` will result in a model with ``"emergent"`` at it's root.

### BiomodalPacmanEmission

The second flavour of the Pacman model is the ``BimodalPacmanEmission`` model which is the same as ``PacmanEmission`` in every other respect but does differentiate between old and young populations. The young component feels attenuation from both the ISM and birth cloud while the old component only feels attenuation from the ISM.

In [ ]:
from synthesizer.emission_models import BimodalPacmanEmission

pc_model = BimodalPacmanEmission(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-1.3),
    dust_curve_nebular=PowerLaw(slope=-0.7),
    dust_emission_ism=Blackbody(temperature=100 * kelvin),
    dust_emission_nebular=Blackbody(temperature=30 * kelvin),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)
pc_model.plot_emission_tree(fontsize=5)

And as with the other models the dust emission can omitted leaving ``"emergent"`` at the root.

## AGN Emission Models

You are, of course, free to construct whatever emission model you want using the preexisting AGN grids but we define a set of premade models including a complex `UnifiedAGN` model to calculate and combine emission from an AGN's disc, Narrow Line Region (NLR), Broad Line Region (BLR), and torus.

As with the stellar models, before we define any AGN models we need the grids to use with them. For AGN models there are currently two grids, one for the NLR and one for the BLR. 

In [ ]:
from synthesizer import Grid

# Get the NLR and BLR grids
nlr_grid = Grid("test_grid_agn-nlr", grid_dir="../../../tests/test_grid")
blr_grid = Grid("test_grid_agn-blr", grid_dir="../../../tests/test_grid")

### NLRIncidentEmission

The incident NLR emission is the pure disc emission incident onto the NLR. A NLR incident model defines the extraction of the ``"incident"`` NLR spectra.

In [ ]:
from synthesizer.emission_models import NLRIncidentEmission

nlr_incident = NLRIncidentEmission(grid=nlr_grid)
print(nlr_incident)

### BLRIncidentEmission 

The incident BLR emission is the pure disc emission incident onto the BLR. A BLR incident model defines the extraction of the ``"incident"`` BLR spectra.

In [ ]:
from synthesizer.emission_models import BLRIncidentEmission

blr_incident = BLRIncidentEmission(grid=blr_grid)
print(blr_incident)

### NLRTransmittedEmission

The transmitted NLR emission is the incident NLR emission which is transmitted through the NLR. A NLR transmitted model defines the extraction of the ``"transmitted"`` NLR spectra including a covering fraction (escape fraction) defining how much of the disc emission is transmitted through the NLR.

In [ ]:
from synthesizer.emission_models import NLRTransmittedEmission

nlr_transmitted = NLRTransmittedEmission(grid=nlr_grid, covering_fraction=0.1)
print(nlr_transmitted)

### BLRTransmittedEmission

The transmitted BLR emission is the incident BLR emission which is transmitted through the BLR. A BLR transmitted model defines the extraction of the ``"transmitted"`` BLR spectra including a covering fraction (escape fraction) defining how much of the disc emission is transmitted through the BLR.

In [ ]:
from synthesizer.emission_models import BLRTransmittedEmission

blr_transmitted = BLRTransmittedEmission(grid=blr_grid, covering_fraction=0.1)
print(blr_transmitted)

### NLREmission

The NLR emission is the emission coming directly from the NLR. A NLR model defines the extraction of the ``"nebular"`` NLR spectra.

In [ ]:
from synthesizer.emission_models import NLREmission

nlr = NLREmission(grid=nlr_grid)
print(nlr)

### BLREmission

The BLR emission is the emission coming directly from the BLR. A BLR model defines the extraction of the ``"nebular"`` BLR spectra.

In [ ]:
blr = NLREmission(grid=blr_grid)
print(blr)

### DiscIncidentEmission

The disc incident emission is the emission directly from the disc incident onto the NLR and BLR. A disc incident model defines the extraction of the ``"incident"`` NLR spectra. Without any geometry considerations ``DiscIncidentEmission``, ``NLRIncidentEmission``, and ``BLRIncidentEmission`` are all equivalent models (we consider the geometry in the ``UnifiedAGN`` model).

In [ ]:
from synthesizer.emission_models import DiscIncidentEmission

disc_incident = DiscIncidentEmission(grid=nlr_grid)
print(disc_incident)


### DiscTransmittedEmission

The disc transmitted emission is the disc emission transmitted through both the NLR and BLR. A disc transmitted model defines the combination of ``NLRTransmittedEmission`` and ``BLRTransmittedEmission`` in the presence of a covering fraction (escape fraction) for each line region.

In [ ]:
from synthesizer.emission_models import DiscTransmittedEmission

disc_transmitted = DiscTransmittedEmission(
    nlr_grid=nlr_grid,
    blr_grid=blr_grid,
    covering_fraction_nlr=0.1,
    covering_fraction_blr=0.2,
)
print(disc_transmitted)
disc_transmitted.plot_emission_tree()

### DiscEscapedEmission 

The disc escaped emission is the disc incident emission *not* transmitted through the NLR or BLR. A disc escaped model defines the extraction of the ``"incident"`` NLR spectra with ``fesc=1 - covering_fraction_nlr - covering_fraction_blr``.

In [ ]:
from synthesizer.emission_models import DiscEscapedEmission

disc_escaped = DiscEscapedEmission(
    grid=nlr_grid, covering_fraction_nlr=0.1, covering_fraction_blr=0.2
)
print(disc_escaped)

### DiscEmission

The disc emission is the combined emission from the disc including both the emission transmitted through the line regions and the escaping disc emission. A disc model defines the combination of ``DiscTransmittedEmission`` and ``DiscEscapedEmission``.

In [ ]:
from synthesizer.emission_models import DiscEmission

disc = DiscEmission(
    nlr_grid=nlr_grid,
    blr_grid=blr_grid,
    covering_fraction_nlr=0.1,
    covering_fraction_blr=0.2,
)
print(disc)
disc.plot_emission_tree()

### TorusEmission 

The torus emission is the emission from the torus due to the incident disc emission is reemits. A torus model defines the generation of a spectra scaled by the `DiscIncidentEmission` which can either be passed, or a grid must be passed and it will be generated.

In [ ]:
from synthesizer.emission_models import TorusEmission

torus = TorusEmission(
    torus_emission_model=Blackbody(1000 * kelvin), disc_incident=disc_incident
)
print(torus)

### AGNIntrinsicEmission 

The AGN intrinsic emission is the total emission from an AGN including the emission from the line regions, disc, and torus. A AGN intrinsic model defines the combination of ``DiscEmission``, ``NLREmission``, ``BLREmission``, and ``TorusEmission``.

In [ ]:
from synthesizer.emission_models import AGNIntrinsicEmission

agn_intrinsic = AGNIntrinsicEmission(
    nlr_grid=nlr_grid,
    blr_grid=blr_grid,
    torus_emission_model=Blackbody(1000 * kelvin),
    covering_fraction_nlr=0.1,
    covering_fraction_blr=0.2,
)
print(agn_intrinsic)
agn_intrinsic.plot_emission_tree()

### UnifiedAGN

The ``UnifiedAGN`` model is similar to the ``AGNIntrinsicEmission`` model but folds in a lot of extra considerations about the geometry of an AGN.

- Disc emission takes into account the inclination of the black hole relative to the angular size of the torus, scaling the amount of observable emission.
- The line region emissions are assumed to be completely isotropic.
- The torus sees the isotropic emission from the disc.
- Only a percentage of the disc emission is transmitted through the line regions.

In [ ]:
from synthesizer.emission_models import UnifiedAGN

uni_model = UnifiedAGN(
    nlr_grid,
    blr_grid,
    covering_fraction_nlr=0.1,
    covering_fraction_blr=0.1,
    torus_emission_model=Blackbody(1000 * kelvin),
)
print(uni_model)

uni_model.plot_emission_tree(fontsize=7)

## Common Models

Some models are common to all components. While these common models are the same in almost every respect these models do require an extra argument to define the "emitter" (``"stellar"``, ``"blackhole"``, or ``"galaxy"``) that they act on.

### TemplateEmission

A template is a simple generation model where a single template spectra is returned with some scaling applying. These can describe any emitter based on the spectra input into the template (which can either come from arrays of ``lnu`` and ``lam`` or a file). 

Generation from a template model requires a ``Template``, similar to the ``Grid`` required by extraction models.

In [ ]:
from synthesizer.emission_models import TemplateEmission
from synthesizer.grid import Template

# Make a fake template for the demo
template = Template(
    lam=grid.lam, lnu=np.random.rand(*grid.lam.shape) * erg / s / Hz
)

template_model = TemplateEmission(template, emitter="stellar")
print(template_model)


### AttenuatedEmission

Attenuated emission is any emission that has been attenuated by dust. An attenuated model takes a ``dust_curve`` (with a defined slope), an optical depth (``tau_v``) which can either be a value, array, attribute of an emitter or tuple containing any combination of the first 3, and a model to apply the dust attenuation to (``apply_dust_to``).

In [ ]:
from synthesizer.emission_models import AttenuatedEmission

attenuated = AttenuatedEmission(
    emitter="stellar",
    dust_curve=PowerLaw(slope=-1),
    tau_v=(0.5, "tau_v"),
    apply_dust_to=intrinsic,
)
print(attenuated)

### DustEmission

Dust emission is the reemited emission from the dust distribution due to the attenuation of another emission source. A dust emission model defines the generation of a spectra from a generator (e.g. ``Blackbody`` or ``Greybody``, see the [dust emission model docs](dust_emission.ipynb)) and it its scaling using an energy balance approach.

In [ ]:
from synthesizer.emission_models import DustEmission

dust_model = DustEmission(
    dust_emission_model=Blackbody(50 * kelvin),
    dust_lum_intrinsic=intrinsic,
    dust_lum_attenuated=attenuated,
    emitter="stellar",
)
print(dust_model)